In [1]:
# Models and metrics
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor
import xgboost as xgb
from hyperopt import hp
import vectorbtpro as vbt

# Classification metrics
from sklearn.metrics import classification_report, precision_recall_curve, auc, r2_score
from sklearn.calibration import calibration_curve, CalibrationDisplay
from sklearn.metrics import f1_score, precision_score, recall_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder

# Suppress all warnings
import warnings

warnings.filterwarnings('ignore')

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Other imports
import pandas as pd
import numpy as np
import joblib
import os
from utils.db_utils import QUERY

ModuleNotFoundError: No module named 'utils'

In [ ]:
ml_features = QUERY('SELECT * FROM (DESCRIBE market_data.ml_features)')['column_name'].values.tolist()

In [ ]:
# Columns we need to drop before training the model
forward_returns_cols = [col for col in ml_features if 'forward_returns' in col]

non_numeric_cols = [
    'asset_id_base','asset_id_base_x','asset_id_base_y', 
    'asset_id_quote','asset_id_quote_x', 'asset_id_quote_y', 
    'exchange_id','exchange_id_x','exchange_id_y', 
    'day_of_week', 'month', 'symbol_id'
]

other_cols = [
    'open_spot', 'high_spot', 'low_spot', 'close_spot', 'volume_spot', 'trades_spot',
    'open_futures', 'high_futures', 'low_futures', 'close_futures', 'volume_futures', 'trades_futures',
    'time_period_end'
]

num_cols = [col for col in ml_features if 'num' in col and 'rz' not in col and 'zscore' not in col and 'percentile' not in col]

dollar_cols = [col for col in ml_features if 'dollar' in col and 'rz' not in col and 'zscore' not in col and 'percentile' not in col]

delta_cols = [col for col in ml_features if 'delta' in col and 'rz' not in col and 'zscore' not in col and 'percentile' not in col]

other = [col for col in ml_features if '10th_percentile' in col or '90th_percentile' in col]

cols_to_drop = (
    forward_returns_cols +
    non_numeric_cols +
    other_cols +
    num_cols +
    dollar_cols +
    delta_cols +
    other
)

# Columns to include in the model
returns_cols = [col for col in ml_features if ('spot_returns' in col or 'futures_returns' in col) and 'cs_' not in col]
returns_cs_cols = [col for col in ml_features if ('spot_returns' in col or 'futures_returns' in col) and 'cs_' in col and 'kurtosis' not in col]

alpha_beta_cols = [col for col in ml_features if ('alpha' in col or 'beta' in col) and 'cs_' not in col]
alpha_beta_cs_cols = [col for col in ml_features if ('alpha' in col or 'beta' in col) and 'cs_' in col and 'kurtosis' not in col]

basis_pct_cols = [col for col in ml_features if 'basis_pct' in col and 'cs_' not in col]
basis_pct_cs_cols = [col for col in ml_features if 'basis_pct' in col and 'cs_' in col and 'kurtosis' not in col]

trade_imbalance_cols = [col for col in ml_features if 'trade_imbalance' in col and 'cs_' not in col]
trade_imbalance_cs_cols = [col for col in ml_features if 'trade_imbalance' in col and 'cs_' in col and 'kurtosis' not in col]

valid_cols = (
    returns_cols +
    returns_cs_cols +
    alpha_beta_cols +
    alpha_beta_cs_cols +
    basis_pct_cols +
    basis_pct_cs_cols +
    trade_imbalance_cols +
    trade_imbalance_cs_cols
)

rz_cols = [col for col in ml_features if ('rz' in col or 'zscore' in col or ('percentile' in col and '10th_percentile' not in col and '90th_percentile' not in col) or col in valid_cols) and 'forward_returns' not in col]

In [ ]:
def run_hyperparameter_tuning(X, y, param_space, max_evals=5, direction = 'long'):
    from hyperopt import fmin, tpe, Trials

    def objective(params):
      params['max_depth'] = int(params['max_depth'])  # Ensure max_depth is an integer
      params['n_estimators'] = int(params['n_estimators'])
      params['min_child_weight'] = int(params['min_child_weight'])  # Ensure min_child_weight is an integer
      
      # Split the data into training and validation sets (e.g., 80% train, 20% validation)
      train_end_date = X['time_period_end'].quantile(0.8)
      X_train = X[X['time_period_end'] <= train_end_date]
      X_val = X[X['time_period_end'] > train_end_date]
      if direction == 'long':
        y_train = (X_train['forward_returns_7'] > 0).astype(int)
        y_val = (X_val['forward_returns_7'] > 0).astype(int)
      else:
        y_train = (X_train['futures_forward_returns_7'] < 0).astype(int)
        y_val = (X_val['futures_forward_returns_7'] < 0).astype(int)

      print(f'Train date range: {X_train["time_period_end"].min()}-{X_train["time_period_end"].max()}')
      print(f'Validation date range: {X_val["time_period_end"].min()}-{X_val["time_period_end"].max()}')
      print()

      # Fit the XGBoost model with the given hyperparameters
      model = xgb.XGBClassifier(**params)
      model.fit(X_train.drop(cols_to_drop, axis=1, errors='ignore'), y_train, eval_set=[(X_val.drop(cols_to_drop, axis=1, errors='ignore'), y_val)])

      # Make predictions on the validation set
      y_pred_proba = model.predict_proba(X_val.drop(cols_to_drop, axis=1, errors='ignore'))[:, 1]
      y_pred = (y_pred_proba >= 0.7).astype(int)
      f1 = f1_score(y_val, y_pred)

      # Calculate Sortino-like metric of 7-day forward returns
      pred_mask = (y_pred == 1)
      forward_returns = X_val['forward_returns_7']
      expectancy = np.mean(forward_returns[pred_mask])
      std_dev_neg = np.std(forward_returns[pred_mask & (forward_returns < 0)])
      sortino_like_metric = (expectancy / std_dev_neg) if std_dev_neg != 0 else 0
      market_returns = X_val['forward_returns_7']
      market_expectancy = np.mean(market_returns)
      market_std_dev_neg = np.std(market_returns[market_returns < 0])
      
      # Calculate classification metrics
      print(f'Hyperparameters: {params}')
      print()
      print(classification_report(y_val, y_pred))
      print()
      print(f'Expectancy: {expectancy}, Market Expectancy: {market_expectancy}')
      print(f'Std Dev Negative: {std_dev_neg}, Market Std Dev Negative: {market_std_dev_neg}')
      print(f'Sortino-Like: {sortino_like_metric}, Market Sortino-Like: {market_expectancy / market_std_dev_neg if market_std_dev_neg != 0 else 0}')
      print()

      # Calculate the loss (negative Sortino-like metric)
      optimization_metric = -f1
      print(f'Optimization Metric: {optimization_metric}')
      print('=' * 80)
      return {'loss': optimization_metric, 'status': 'ok'}

    trials = Trials()
    best = fmin(objective, param_space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    return best

In [ ]:
def train_model(min_year, max_year, is_reg, model_type, direction='long'):
    for year in range(min_year, max_year + 1):
        # Train XGBoost model for each month
        for month in range(1, 13):
            # if year < 2022 or (year == 2022 and month < 6):
            #     continue
            
            if direction == 'long':
                query = f"""
                SELECT *
                FROM market_data.ml_features
                WHERE
                    date_part('year', time_period_end) < {year} OR
                    (date_part('year', time_period_end) = {year} AND
                     date_part('month', time_period_end) <= {month})
                """
            else:
                query = f"""
                SELECT *
                FROM market_data.ml_features
                WHERE
                    date_part('year', time_period_end) < {year} OR
                    (date_part('year', time_period_end) = {year} AND
                     date_part('month', time_period_end) <= {month}) AND
                     close_futures IS NOT NULL -- Ensure futures data is available for the model to train
                """

            data_train = QUERY(query)
            data_train['symbol_id'] = (
                data_train['asset_id_base'].str.upper() +
                '_' +
                data_train['asset_id_quote'].str.upper() +
                '_' +
                data_train['exchange_id'].str.upper()
            ).astype('category')
            data_train['day_of_week'] = data_train['day_of_week'].astype('category')
            data_train['month'] = data_train['month'].astype('category')

            max_train_date = pd.to_datetime(data_train['time_period_end'].dt.date.max())
            min_train_date = pd.to_datetime(max_train_date - pd.Timedelta(days = 365 * 2))

            # Test data is all data in the next month
            next_month = month + 1
            if next_month == 13:
                next_year = year + 1
                next_month = 1
            else:
                next_year = year

            if direction == 'long':
                query = f"""
                SELECT *
                FROM market_data.ml_features
                WHERE
                    time_period_end > '{max_train_date}' AND
                    date_part('year', time_period_end) = {next_year} AND
                    date_part('month', time_period_end) <= {next_month}
                """
            else:
                query = f"""
                SELECT *
                FROM market_data.ml_features
                WHERE
                    time_period_end > '{max_train_date}' AND
                    date_part('year', time_period_end) = {next_year} AND
                    date_part('month', time_period_end) <= {next_month} AND
                    close_futures IS NOT NULL -- Ensure futures data is available for the model to test
                """
            data_test = QUERY(query)
            data_test['symbol_id'] = (
                data_test['asset_id_base'].str.upper() +
                '_' +
                data_test['asset_id_quote'].str.upper() +
                '_' +
                data_test['exchange_id'].str.upper()
            ).astype('category')
            data_test['day_of_week'] = data_test['day_of_week'].astype('category')
            data_test['month'] = data_test['month'].astype('category')

            data_train.replace([np.inf, -np.inf], np.nan, inplace=True)
            data_test.replace([np.inf, -np.inf], np.nan, inplace=True)

            # Sort data by symbol_id and time_period_end
            data_train = data_train.sort_values(by=['symbol_id', 'time_period_end'])
            data_test = data_test.sort_values(by=['symbol_id', 'time_period_end'])

            if direction == 'long':
                # Filter out data with nan forward returns
                data_train = data_train.dropna(subset=['forward_returns_7'])
                data_test = data_test.dropna(subset=['forward_returns_7'])
            else:
                # Filter out data with nan futures forward returns
                data_train = data_train.dropna(subset=['futures_forward_returns_7'])
                data_test = data_test.dropna(subset=['futures_forward_returns_7'])

            # Filter out training data older than 2 years from the max train date
            filter_train = (
                data_train['time_period_end'] >= min_train_date
            )
            filter_test = (
            )
            # data_train = data_train[filter_train]
            # data_test = data_test[filter_test]

            data_train['symbol_id'] = data_train['symbol_id'].astype('category')
            data_test['symbol_id'] = data_test['symbol_id'].astype('category')

            if data_train.empty or data_test.empty:
                continue

            X_train = data_train
            X_test = data_test

            # Ensure no data leakage
            assert X_train['time_period_end'].max() < X_test['time_period_end'].min(), 'Data leakage detected'

            # Split data into features and target
            if is_reg:
                y_train = X_train['forward_returns_7'].abs()
                y_test = X_test['forward_returns_7'].abs()
            else:
                if direction == 'long':
                    y_train = (X_train['forward_returns_7'] > 0).astype(int)
                    y_test = (X_test['forward_returns_7'] > 0).astype(int)
                else:
                    y_train = (X_train['futures_forward_returns_7'] < 0).astype(int)
                    y_test = (X_test['futures_forward_returns_7'] < 0).astype(int)

            print()
            print(f'Train Date Range: {X_train["time_period_end"].min()} - {X_train["time_period_end"].max()}')
            print(f'Number of observations (Train): {X_train.shape[0]}')
            print()

            print(f'Test Date Range: {X_test["time_period_end"].min()} - {X_test["time_period_end"].max()}')
            print(f'Number of observations (Test): {X_test.shape[0]}')
            print()

            # Ensure no data leakage
            data_leakage_indicator = (
                (X_train['time_period_end'].max() >= X_test['time_period_end'].min())
            )
            assert not data_leakage_indicator, 'Data leakage detected'

            if model_type != 'xgb':
                # Define the model
                ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

                # OHE train data
                encoded_data = ohe.fit_transform(X_train[['symbol_id', 'day_of_week', 'month']])
                encoded_cols = ohe.get_feature_names_out(['symbol_id', 'day_of_week', 'month'])
                encoded_df = pd.DataFrame(encoded_data, columns = encoded_cols, index = X_train.index)
                X_train = pd.concat([X_train[rz_cols], encoded_df], axis = 1)

                # OHE test data
                encoded_data = ohe.transform(X_test[['symbol_id', 'day_of_week', 'month']])
                encoded_cols = ohe.get_feature_names_out(['symbol_id', 'day_of_week', 'month'])
                encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index = X_test.index)
                X_test_ = pd.concat([X_test_[rz_cols], encoded_df], axis = 1)

            # Winsorize training and test data
            # Ensure no NaN values in the training and test data if not using XGBoost
            if model_type != 'xgb':
                p001 = X_train.quantile(0.001)
                p999 = X_train.quantile(0.999)
                X_train = X_train.clip(lower=p001, upper=p999, axis=1)
                # Winsorize test data using train percentiles
                X_test_ = X_test_.clip(lower=p001, upper=p999, axis=1)

                X_train = X_train.fillna(0)
                X_test_ = X_test_.fillna(0)
            
            # Define the model
            if model_type == 'lr':
                model = LogisticRegression(
                    penalty='elasticnet',
                    solver='saga',
                    l1_ratio=0.5,
                    class_weight='balanced',
                    C = 0.1,
                    random_state=9+10,
                    n_jobs=-1,
                )
            elif model_type == 'nn':
                model = MLPClassifier(
                    hidden_layer_sizes=(100,100), 
                    alpha = 10, 
                    verbose=True, 
                    max_iter = 100, 
                    random_state = 9+10, 
                    tol = 0.01, 
                    n_iter_no_change=1
                )
            elif model_type == 'xgb':
                param_space = {
                    'objective': 'binary:logistic',
                    'n_estimators': 200,
                    'max_depth': 12,
                    'min_child_weight': 5,
                    'reg_lambda': 10,
                    'random_state': 9+10,
                    'class_weight': 'balanced',
                    'njobs': -1
                }
                model = xgb.XGBClassifier(**param_space)
            
            if direction == 'long':
                y_train = (X_train['forward_returns_7'] > 0).astype(int)
                y_test = (X_test['forward_returns_7'] > 0).astype(int)
            else:
                y_train_ = (X_train['futures_forward_returns_7'] < 0).astype(int)
                y_test = (X_test['futures_forward_returns_7'] < 0).astype(int)

            futures_forward_returns_cols = [col for col in X_train.columns if 'forward_returns' in col]

            X_train = X_train.drop(columns=cols_to_drop + futures_forward_returns_cols, errors='ignore', axis=1)
            X_test_ = X_test.drop(columns=cols_to_drop + futures_forward_returns_cols, errors='ignore', axis=1)

            if model_type != 'xgb':
                model.fit(X_train, y_train)
            else:
                model.fit(
                    X_train, 
                    y_train
                )

            y_pred_proba = model.predict_proba(X_test_)[:, 1]
            y_pred = (y_pred_proba >= 0.7).astype(int)

            X_test['y_true'] = y_test
            X_test['y_pred'] = y_pred
            X_test['y_pred_proba'] = y_pred_proba

            print('Class Distribution:')
            print(X_test['y_true'].value_counts(normalize = True))
            print()

            trade_side = np.where(
                y_pred == 1, 1, 0
            )
            if direction == 'short':
                trade_side = np.where(y_pred == 1, -1, 0)
                trade_pnl = trade_side * X_test['futures_forward_returns_7'].values
                trades_mask = trade_side == -1
                expectancy = trade_pnl[trades_mask].mean()
                std_neg = trade_pnl[trades_mask & (trade_pnl < 0)].std()
                sortino_like = expectancy / std_neg if std_neg != 0 else 0
                hit_rate = (trade_pnl[trades_mask] > 0).mean()
            else:
                trade_side = np.where(y_pred == 1, 1, 0)
                trade_pnl = trade_side * X_test['forward_returns_7'].values
                trades_mask = trade_side == 1
                expectancy = trade_pnl[trades_mask].mean()
                std_neg = trade_pnl[trades_mask & (trade_pnl < 0)].std()
                sortino_like = expectancy / std_neg if std_neg != 0 else 0
                hit_rate = (trade_pnl[trades_mask] > 0).mean()

            if direction == 'long':
                # Market Performance
                market_expectancy = X_test['forward_returns_7'].mean()
                market_std_neg = X_test['forward_returns_7'][X_test['forward_returns_7'] < 0].std()
                market_sortino_like = market_expectancy / market_std_neg if market_std_neg != 0 else 0
                market_hit_rate = (X_test['forward_returns_7'] > 0).mean()
            else:
                # Market Performance
                market_expectancy = -X_test['futures_forward_returns_7'].mean()
                market_std_neg = X_test['futures_forward_returns_7'][X_test['futures_forward_returns_7'] > 0].std()
                market_sortino_like = market_expectancy / market_std_neg if market_std_neg != 0 else 0
                market_hit_rate = (X_test['futures_forward_returns_7'] < 0).mean()
                
            print(f'Expectancy: {expectancy:.3f}, Market Expectancy: {market_expectancy:.3f}')
            print(f'Std Negative Returns: {std_neg:.3f}, Market Std Negative Returns: {market_std_neg:.3f}')
            print(f'Sortino-Like: {sortino_like:.3f}, Market Sortino-Like: {market_sortino_like:.3f}')
            print(f'Hit Rate: {hit_rate:.3f}, Market Hit Rate: {market_hit_rate:.3f}')
            print()

            # Classification Report
            print('Classification Report:')
            print(classification_report(X_test['y_true'], X_test['y_pred']))
            print()

            # Calibration Curve
            disp = CalibrationDisplay.from_predictions(y_test, y_pred_proba)
            plt.show()

            precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
            pr_auc = auc(recall, precision)

            if model_type == 'lr':
                # Subplots 2 columns, 1 row
                fig, ax = plt.subplots(1, 2, figsize=(15, 7))

                # Top 50 most important positive features
                top_n_pos = 50
                feature_importances = pd.Series(model.coef_[0], index=X_test_.columns)
                feature_importances = feature_importances.sort_values().tail(top_n_pos)
                feature_importances.plot(kind='barh', title='Top 50 Most Important Features', ax=ax[0])

                # Top 50 most important negative features
                top_n_neg = 50
                feature_importances_neg = pd.Series(model.coef_[0], index=X_test_.columns)
                feature_importances_neg = feature_importances_neg.sort_values().head(top_n_neg)
                feature_importances_neg.plot(kind='barh', title='Top 50 Most Important Negative Features', ax=ax[1])

                plt.tight_layout()
                plt.show()
            elif model_type == 'xgb':
                try:
                    fig, ax = plt.subplots(figsize=(15, 7))
                    # Plot feature importance
                    xgb.plot_importance(model, max_num_features=50, importance_type='gain', ax=ax, title='Top 50 Most Important Features')
                    plt.show()
                except Exception as e:
                    print(f'Error plotting feature importance: {e}')
                    print()
            
            # Delete old data from memory
            del X_train
            del X_test
            del data_train
            del data_test

            if model_type == 'lr':
                # Save the model
                model_path = f'/Users/louisspencer/Desktop/Trading-Bot/data/pretrained_models/classification/lr_model_{next_year}_{next_month}.pkl'
                joblib.dump(model, model_path)
            elif model_type == 'nn':
                # Save the model
                model_path = f'/Users/louisspencer/Desktop/Trading-Bot/data/pretrained_models/classification/nn_model_{next_year}_{next_month}.pkl'
                joblib.dump(model, model_path)
            elif model_type == 'xgb':
                
                if direction == 'long':
                    model_path = f'/Users/louisspencer/Desktop/Trading-Bot/data/pretrained_models/classification/xgb_long_model_{next_year}_{next_month}.pkl'
                else:
                    model_path = f'/Users/louisspencer/Desktop/Trading-Bot/data/pretrained_models/classification/xgb_short_model_{next_year}_{next_month}.pkl'
                
                # Save the model
                joblib.dump(model, model_path)

            if model_type != 'xgb':
                # Save the OneHotEncoder to have the same encoding for backtesting
                ohe_path = f'/Users/louisspencer/Desktop/Trading-Bot/data/pretrained_models/classification/ohe_{next_year}_{next_month}.pkl'
                joblib.dump(ohe, ohe_path)

In [2]:
train_model(2020, 2025, is_reg=False, model_type='xgb', direction='short')

NameError: name 'train_model' is not defined

In [ ]:
def plot_performance_metrics(min_year, max_year, is_reg, model_type, direction='long'):
    performance_metrics = []
    init_cash = 10_000
    for year in range(min_year, max_year + 1):
        # Train XGBoost model for each month
        for month in range(1, 13):
            if year == 2018 and month < 11:
                continue

            if direction == 'long':
                data_train = QUERY(
                    f"""
                    SELECT *
                    FROM market_data.ml_features
                    WHERE
                        date_part('year', time_period_end) < {year} OR
                        (date_part('year', time_period_end) = {year} AND
                        date_part('month', time_period_end) <= {month})
                    """
                )
            else:
                data_train = QUERY(
                    f"""
                    SELECT *
                    FROM market_data.ml_features
                    WHERE
                        date_part('year', time_period_end) < {year} OR
                        (date_part('year', time_period_end) = {year} AND
                        date_part('month', time_period_end) <= {month}) AND
                        close_futures IS NOT NULL -- Ensure futures data is available for the model to train
                    """
                )
            data_train['symbol_id'] = (
                data_train['asset_id_base'].str.upper() +
                '_' +
                data_train['asset_id_quote'].str.upper() +
                '_' +
                data_train['exchange_id'].str.upper()
            ).astype('category')
            data_train['day_of_week'] = data_train['day_of_week'].astype('category')
            data_train['month'] = data_train['month'].astype('category')

            max_train_date = pd.to_datetime(data_train['time_period_end'].dt.date.max())
            min_train_date = pd.to_datetime(max_train_date - pd.Timedelta(days = 365 * 2))

            # Test data is all data in the next month
            next_month = month + 1
            if next_month == 13:
                next_year = year + 1
                next_month = 1
            else:
                next_year = year

            if direction == 'long':
                data_test = QUERY(
                    f"""
                    SELECT *
                    FROM market_data.ml_features
                    WHERE
                        time_period_end > '{max_train_date}' AND
                        date_part('year', time_period_end) = {next_year} AND
                        date_part('month', time_period_end) <= {next_month}
                    """
                )
            else:
                data_test = QUERY(
                    f"""
                    SELECT *
                    FROM market_data.ml_features
                    WHERE
                        time_period_end > '{max_train_date}' AND
                        date_part('year', time_period_end) = {next_year} AND
                        date_part('month', time_period_end) <= {next_month} AND
                        close_futures IS NOT NULL -- Ensure futures data is available for the model to test
                    """
                )
            data_test['symbol_id'] = (
                data_test['asset_id_base'].str.upper() +
                '_' +
                data_test['asset_id_quote'].str.upper() +
                '_' +
                data_test['exchange_id'].str.upper()
            ).astype('category')
            data_test['day_of_week'] = data_test['day_of_week'].astype('category')
            data_test['month'] = data_test['month'].astype('category')

            data_train.replace([np.inf, -np.inf], np.nan, inplace=True)
            data_test.replace([np.inf, -np.inf], np.nan, inplace=True)

            # Filter out data with nan trade_returns_h7
            data_train = data_train.dropna(subset = ['forward_returns_7'])
            data_test = data_test.dropna(subset = ['forward_returns_7'])

            # Filter out training data older than 2 years from the max train date
            filter_train = (
                data_train['time_period_end'] >= min_train_date
            )
            filter_test = (
            )
            data_train = data_train[filter_train]

            if data_train.empty or data_test.empty:
                continue

            X_train = data_train
            X_test = data_test

            # Split data into features and target
            if is_reg:
                y_train = X_train['forward_returns_7'].abs()
                y_test = X_test['forward_returns_7'].abs()
            else:
                if direction == 'long':
                    y_train = (X_train['forward_returns_7'] > 0).astype(int)
                    y_test = (X_test['forward_returns_7'] > 0).astype(int)
                else:
                    y_train = (X_train['futures_forward_returns_7'] < 0).astype(int)
                    y_test = (X_test['futures_forward_returns_7'] < 0).astype(int)

            print()
            print(f'Train Date Range: {X_train["time_period_end"].min()} - {X_train["time_period_end"].max()}')
            print(f'Number of observations (Train): {X_train.shape[0]}')
            print()

            print(f'Test Date Range: {X_test["time_period_end"].min()} - {X_test["time_period_end"].max()}')
            print(f'Number of observations (Test): {X_test.shape[0]}')
            print()

            # Ensure no data leakage
            data_leakage_indicator = (
                (X_train['time_period_end'].max() > X_test['time_period_end'].min())
            )
            assert not data_leakage_indicator, 'Data leakage detected'

            X_train = X_train.drop(columns=cols_to_drop)
            X_test_ = X_test.drop(columns=cols_to_drop)

            if model_type != 'xgb':
                # Define the model
                ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

                # OHE train data
                encoded_data = ohe.fit_transform(X_train[['symbol_id', 'day_of_week', 'month']])
                encoded_cols = ohe.get_feature_names_out(['symbol_id', 'day_of_week', 'month'])
                encoded_df = pd.DataFrame(encoded_data, columns = encoded_cols, index = X_train.index)
                X_train = pd.concat([X_train[rz_cols], encoded_df], axis = 1)

                # OHE test data
                encoded_data = ohe.transform(X_test[['symbol_id', 'day_of_week', 'month']])
                encoded_cols = ohe.get_feature_names_out(['symbol_id', 'day_of_week', 'month'])
                encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index = X_test.index)
                X_test_ = pd.concat([X_test_[rz_cols], encoded_df], axis = 1)

            # Winsorize training and test data
            # Ensure no NaN values in the training and test data if not using XGBoost
            if model_type != 'xgb':
                p001 = X_train.quantile(0.001)
                p999 = X_train.quantile(0.999)
                X_train = X_train.clip(lower=p001, upper=p999, axis=1)
                # Winsorize test data using train percentiles
                X_test_ = X_test_.clip(lower=p001, upper=p999, axis=1)

                X_train = X_train.fillna(0)
                X_test_ = X_test_.fillna(0)
            
            # Define the model
            if model_type == 'lr':
                model_path = f'/Users/louisspencer/Desktop/Trading-Bot/data/pretrained_models/classification/lr_model_{next_year}_{next_month}.pkl'
            elif model_type == 'nn':
                model_path = f'/Users/louisspencer/Desktop/Trading-Bot/data/pretrained_models/classification/nn_model_{next_year}_{next_month}.pkl'
            elif model_type == 'xgb':
                if direction == 'long':
                    model_path = f'/Users/louisspencer/Desktop/Trading-Bot/data/pretrained_models/classification/xgb_long_model_{next_year}_{next_month}.pkl'
                else:
                    model_path = f'/Users/louisspencer/Desktop/Trading-Bot/data/pretrained_models/classification/xgb_short_model_{next_year}_{next_month}.pkl'

            model = joblib.load(model_path)
            y_pred_proba = model.predict_proba(X_test_)[:, 1]
            y_pred = (y_pred_proba >= 0.7).astype(int)

            X_test['y_true'] = y_test
            X_test['y_pred'] = y_pred
            X_test['y_pred_proba'] = y_pred_proba

            # Create cross-sectional predictions
            pivot_preds = X_test.pivot_table(index = 'time_period_end', columns = 'symbol_id', values = 'y_pred', dropna = False).fillna(0)
            if direction == 'long':
                open = X_test.pivot_table(index = 'time_period_end', columns = 'symbol_id', values = 'open_spot', dropna = False)
                high = X_test.pivot_table(index = 'time_period_end', columns = 'symbol_id', values = 'high_spot', dropna = False)
                low = X_test.pivot_table(index = 'time_period_end', columns = 'symbol_id', values = 'low_spot', dropna = False)
                close = X_test.pivot_table(index = 'time_period_end', columns = 'symbol_id', values = 'close_spot', dropna = False)
                long_entries = (pivot_preds == 1).astype(bool)
                short_entries = np.full(pivot_preds.shape, False, dtype=bool)

                market_expectancy = X_test['forward_returns_7'].mean()
                market_std_neg = X_test['forward_returns_7'][X_test['forward_returns_7'] < 0].std()
                market_sortino_like = market_expectancy / market_std_neg if market_std_neg != 0 else 0
                market_hit_rate = (X_test['forward_returns_7'] > 0).mean()

                trade_mask = (y_pred == 1)
                trade_pnl = X_test['forward_returns_7'] * trade_mask
                expectancy = trade_pnl[trade_mask].mean()
                std_neg = trade_pnl[trade_mask & (trade_pnl < 0)].std()
                sortino_like = expectancy / std_neg if std_neg != 0 else 0
                hit_rate = (trade_pnl[trade_mask] > 0).mean()
                
            else:
                open = X_test.pivot_table(index = 'time_period_end', columns = 'symbol_id', values = 'open_futures', dropna = False)
                high = X_test.pivot_table(index = 'time_period_end', columns = 'symbol_id', values = 'high_futures', dropna=False)
                low = X_test.pivot_table(index = 'time_period_end',columns = 'symbol_id',values = 'low_futures',dropna=False)
                close = X_test.pivot_table(index = 'time_period_end',columns = 'symbol_id',values = 'close_futures',dropna=False)
                short_entries = (pivot_preds == 1).astype(bool)
                long_entries = np.full(pivot_preds.shape, False, dtype=bool)

                market_expectancy = -X_test['futures_forward_returns_7'].mean()
                market_std_neg = X_test['futures_forward_returns_7'][X_test['futures_forward_returns_7'] > 0].std()
                market_sortino_like = market_expectancy / market_std_neg if market_std_neg != 0 else 0
                market_hit_rate = (X_test['futures_forward_returns_7'] < 0).mean()

                trade_mask = (y_pred == 1)
                trade_pnl = -X_test['futures_forward_returns_7'] * trade_mask
                expectancy = trade_pnl[trade_mask].mean()
                std_neg = trade_pnl[trade_mask & (trade_pnl < 0)].std()
                sortino_like = expectancy / std_neg if std_neg != 0 else 0
                hit_rate = (trade_pnl[trade_mask] > 0).mean()

            # Print market performance
            print(f'Expectancy: {expectancy:.3f}, Market Expectancy: {market_expectancy:.3f}')
            print(f'Std Negative Returns: {std_neg:.3f}, Market Std Negative Returns: {market_std_neg:.3f}')
            print(f'Sortino-Like: {sortino_like:.3f}, Market Sortino-Like: {market_sortino_like:.3f}')
            print(f'Hit Rate: {hit_rate:.3f}, Market Hit Rate: {market_hit_rate:.3f}')
            print()

            # Simulate portfolio
            pf = vbt.Portfolio.from_signals(
                size_type = 'valuepercent',
                init_cash = init_cash,
                short_entries = short_entries,
                open = open,
                high = high,
                low = low,
                close = close,
                sl_stop = 0.2,
                size = 0.05,
                td_stop = pd.Timedelta(days=7),
                cash_sharing = True,
                accumulate = False,
                freq = 'D'
            )
            pf_stats = pf.stats()
            sharpe = pf_stats['Sharpe Ratio']
            sortino = pf_stats['Sortino Ratio']
            equity_curve = pf.value.to_frame()
            equity_curve = equity_curve.rename({equity_curve.columns[0]:'equity'}, axis = 1)
            final_equity = equity_curve['equity'].iloc[-1]
            backtest_return = (final_equity - init_cash) / init_cash
                        
            # Positions
            rename_dict = {
                'Entry Index':'entry_date', 'Exit Index':'exit_date', 
                'Column':'symbol_id', 'Size':'size', 'Entry Fees':'entry_fees',
                'Exit Fees':'exit_fees', 'PnL':'pnl', 'Return':'pnl_pct', 'Direction':'is_long',
                'Status':'status'
            }
            cols = ['Entry Index', 'Exit Index', 'Column', 'Size', 'Entry Fees', 'Exit Fees', 'PnL', 'Return', 'Direction', 'Status']
            positions = pf.positions.records_readable
            positions = positions[cols]
            positions = positions.rename(rename_dict, axis = 1)
            positions.sort_values(by='entry_date', inplace=True)

            print('Class Distribution:')
            print(X_test['y_true'].value_counts(normalize = True))
            print()

            trade_side = np.where(
                y_pred == 1, 1, 0
            )
            # Model Performance
            hit_rate = (positions['pnl_pct'] > 0).mean()

            performance_metrics.append({
                'year': next_year,
                'month': next_month,
                'sortino_annualized': sortino,
                'sharpe_annualized': sharpe,
                'hit_rate': hit_rate,
                'backtest_return': backtest_return,
            })
            print(f'Initial Equity: {init_cash}, Final Equity: {final_equity:.2f}')
            print(f'Backtest Return: {backtest_return:.2%}')
            print(f'Sharpe: {sharpe:.3f}, Sortino: {sortino:.3f}, Hit Rate: {hit_rate:.3f}')
            print()

            init_cash = final_equity  # Update initial cash for the next month

            # Classification Report
            print('Classification Report:')
            print(classification_report(X_test['y_true'], X_test['y_pred']))
            print()

            # Calibration Curve
            disp = CalibrationDisplay.from_predictions(y_test, y_pred_proba)
            plt.show()

            precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
            pr_auc = auc(recall, precision)

            if model_type == 'lr':
                # Subplots 2 columns, 1 row
                fig, ax = plt.subplots(1, 2, figsize=(15, 7))

                # Top 50 most important positive features
                top_n_pos = 50
                feature_importances = pd.Series(model.coef_[0], index=X_test_.columns)
                feature_importances = feature_importances.sort_values().tail(top_n_pos)
                feature_importances.plot(kind='barh', title='Top 50 Most Important Features', ax=ax[0])

                # Top 50 most important negative features
                top_n_neg = 50
                feature_importances_neg = pd.Series(model.coef_[0], index=X_test_.columns)
                feature_importances_neg = feature_importances_neg.sort_values().head(top_n_neg)
                feature_importances_neg.plot(kind='barh', title='Top 50 Most Important Negative Features', ax=ax[1])

                plt.tight_layout()
                plt.show()
            elif model_type == 'xgb':
                try:
                    fig, ax = plt.subplots(figsize=(15, 7))
                    # Plot feature importance
                    xgb.plot_importance(model, max_num_features=50, importance_type='gain', ax=ax, title='Top 50 Most Important Features')
                    plt.show()
                except Exception as e:
                    print(f'Error plotting feature importance: {e}')
                    print()

            # Delete old data from memory
            del X_train
            del X_test
            del data_train
            del data_test

    # Convert performance metrics to DataFrame
    performance_df = pd.DataFrame(performance_metrics)
    performance_df.to_csv(f'/Users/louisspencer/Desktop/Trading-Bot/data/performance_metrics_{model_type}_{direction}.csv', index=False)

In [ ]:
plot_performance_metrics(2020, 2025, is_reg=False, model_type='xgb', direction='short')

In [ ]:
perf = pd.read_csv('/Users/louisspencer/Desktop/Trading-Bot/notebooks/performance_metrics_xgb.csv')
perf

In [ ]:
pct_month_pos = (perf['backtest_return'] > 0).mean()
expectancy_pos_months = perf[perf['backtest_return'] > 0]
expectancy_neg_months = perf[perf['backtest_return'] <= 0]
expectancy_months_annualized = perf['backtest_return'].mean() 

print(f'Average Monthly Return: {expectancy_months:.3f}')
print(f'Average Return of positive months: {expectancy_pos_months["backtest_return"].mean():.3f}')
print(f'Average Return of negative months: {expectancy_neg_months["backtest_return"].mean():.3f}')
print(f'Average Monthly Sharpe Ratio (Annualized): {perf["sharpe_annualized"].mean():.3f}')
print(f'Average Monthly Sortino Ratio (Annualized): {perf["sortino_annualized"].mean():.3f}')
print(f'Average Monthly Hit Rate: {perf["hit_rate"].mean():.3f}')
print(f'Percentage of months with positive returns: {pct_month_pos:.2%}')

In [ ]:
perf['date'] = pd.to_datetime(perf['year'].astype(str) + '-' + perf['month'].astype(str) + '-01')
perf['month'] = perf['date'].dt.strftime('%Y-%m')

In [ ]:
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Plot monthly backtest returns with Plotly
backtest_res = (perf.set_index('date')['backtest_return'] + 1).cumprod()
fig = px.line(
    backtest_res, 
    title='Cumulative Backtest Returns Over Time',
    labels={'value': 'Cumulative Return', 'date': 'Date'},
    markers=True
)
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Cumulative Return',
    xaxis_rangeslider_visible=True,
    template='plotly_dark'
)
fig.show()

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
perf.head(80)